## Import repository

In [1]:
!git clone https://github.com/federico2879/MLDL2024_semantic_segmentation.git

Cloning into 'MLDL2024_semantic_segmentation'...
remote: Enumerating objects: 552, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 552 (delta 117), reused 150 (delta 93), pack-reused 349
Receiving objects: 100% (552/552), 272.11 KiB | 2.89 MiB/s, done.
Resolving deltas: 100% (324/324), done.


## Import package

In [2]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 791.2 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=b0c620576612aa18b316470daf092f1248221477d76696a0e741d1b32210dc9c
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=dd2f7283ce04d48d46ed9f2088d840857a1b72d19dda7dc51d2e6ee1ad8e5d1a
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [3]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import DataLoader
import numpy as np
from MLDL2024_semantic_segmentation.datasets.cityscapes import CityScapes
from MLDL2024_semantic_segmentation.models.bisenet.build_bisenet import *
from MLDL2024_semantic_segmentation.train import *
from MLDL2024_semantic_segmentation.utils import *
from MLDL2024_semantic_segmentation.models.metrics import *
from MLDL2024_semantic_segmentation.models.IOU import *

## Setup

In [4]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Setup fixed parameters
num_classes = 19
num_epochs = 50
batch_size = 4

cuda


## Dataset

In [5]:
# Transformations
transform_image = transforms.Compose([
    transforms.Resize((1024, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_target = transforms.Compose([
    transforms.Resize((1024, 512), interpolation=InterpolationMode.NEAREST)
])

In [6]:
# Create dataloader

train_dataset = CityScapes('/kaggle/input/Cityscapes/Cityspaces', 
                           split = 'train', transform = transform_image, 
                           label_transform = transform_target)

dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = CityScapes('/kaggle/input/Cityscapes/Cityspaces', 
                         split = 'val', transform = transform_image, 
                         label_transform = transform_target)

dataloader_val = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## Network, Loss, Optimizer

In [7]:
# Inizialization of the model
model = BiSeNet(num_classes=num_classes, context_path="resnet18").to(device)

#Putting on the 2 gpus
model = torch.nn.DataParallel(model, device_ids = [0,1]).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 146MB/s] 
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 166MB/s]  


In [8]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.SGD(model.parameters(), lr=2.5e-2,
                            momentum=0.9,weight_decay=1e-4)

## Training

In [9]:
# metrics

meanIOU_tr = np.zeros((num_epochs,1))
IOU_tr = np.zeros((num_epochs,num_classes))
loss_tr = np.zeros((num_epochs,1))


meanIOU_val = np.zeros((num_epochs,1))
IOU_val = np.zeros((num_epochs,num_classes))
loss_val = np.zeros((num_epochs,1))

In [10]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [11]:
for epoch in range(num_epochs):
    poly_lr_scheduler(optimizer, 2.5e-2, epoch, lr_decay_iter=1,
                      max_iter=num_epochs, power=0.9)
    meanIOU_tr[epoch], IOU_tr[epoch,:], loss_tr[epoch] = train(model, optimizer, dataloader_train, loss_fn, num_classes, 0)
    meanIOU_val[epoch], IOU_val[epoch,:], loss_val[epoch] = test(model, dataloader_val, loss_fn, num_classes, 0)
    print(f"epoch: {epoch + 1}, Validation IOU: {meanIOU_val[epoch,0]:.2f}")

    torch.save({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'meanIOU_tr': meanIOU_tr,
        'IOU_tr': IOU_tr,
        'loss_tr': loss_tr,
        'meanIOU_val': meanIOU_val,
        'IOU_val': IOU_val,
        'loss_val': loss_val
        },"checkpoint.pth.tar")

epoch: 1, Validation IOU: 0.17
epoch: 2, Validation IOU: 0.13
epoch: 3, Validation IOU: 0.23
epoch: 4, Validation IOU: 0.26
epoch: 5, Validation IOU: 0.20
epoch: 6, Validation IOU: 0.28
epoch: 7, Validation IOU: 0.29
epoch: 8, Validation IOU: 0.29
epoch: 9, Validation IOU: 0.30
epoch: 10, Validation IOU: 0.31
epoch: 11, Validation IOU: 0.33
epoch: 12, Validation IOU: 0.32
epoch: 13, Validation IOU: 0.35
epoch: 14, Validation IOU: 0.35
epoch: 15, Validation IOU: 0.35
epoch: 16, Validation IOU: 0.37
epoch: 17, Validation IOU: 0.36
epoch: 18, Validation IOU: 0.38
epoch: 19, Validation IOU: 0.39
epoch: 20, Validation IOU: 0.38
epoch: 21, Validation IOU: 0.40
epoch: 22, Validation IOU: 0.40
epoch: 23, Validation IOU: 0.39
epoch: 24, Validation IOU: 0.40
epoch: 25, Validation IOU: 0.40
epoch: 26, Validation IOU: 0.41
epoch: 27, Validation IOU: 0.40
epoch: 28, Validation IOU: 0.42
epoch: 29, Validation IOU: 0.41
epoch: 30, Validation IOU: 0.41
epoch: 31, Validation IOU: 0.39
epoch: 32, Valida

In [16]:
# Final print

print(f"Final mIOU: {meanIOU_val[epoch,0]:.2f}")
print("Final IOU classes")
print(IOU_val[epoch,:])

flops = Flops(model, 1024, 512, device)

latency = Latency_FPS(model, 1024, 512, device)

print(f"Latency: {latency}")

Final mIOU: 0.40
Final IOU classes
[0.95809125 0.69918359 0.79991891 0.1694542  0.16041384 0.16848095
 0.14150761 0.29490731 0.83447401 0.46275479 0.86507155 0.41605389
 0.0713795  0.80781963 0.06271263 0.17508197 0.10742876 0.08606816
 0.38991113]
| module                                      | #parameters or shape   | #flops     |
|:--------------------------------------------|:-----------------------|:-----------|
| module                                      | 12.582M                | 25.78G     |
|  saptial_path                               |  0.371M                |  5.088G    |
|   saptial_path.convblock1                   |   1.856K               |   0.243G   |
|    saptial_path.convblock1.conv1            |    1.728K              |    0.226G  |
|    saptial_path.convblock1.bn               |    0.128K              |    16.777M |
|   saptial_path.convblock2                   |   73.984K              |   2.424G   |
|    saptial_path.convblock2.conv1            |    73.728K     

In [15]:
# writing csv
import csv

with open('meanIOU_tr.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(meanIOU_tr)

with open('IOU_tr.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(IOU_tr)

with open('loss_tr.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(loss_tr)

with open('meanIOU_val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(meanIOU_val)

with open('IOU_val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(IOU_val)

with open('loss_val.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(loss_val)